# Imports

In [2]:
import numpy as np
import matplotlib.pyplot as plt
# from matplotlib import animation
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Initializing

In [24]:
# Constants and domain setup
k = 1.0             # Wave number
L = 5              # Length of the domain
T = 10               # Total time
nx = 30            # Number of spatial steps
# nt = 200            # Number of temporal steps
dx = L / (nx - 1)   # Spatial step size
#dt = T / nt         # Temporal step size
dt = 1 * dx / k / 2         # Temporal step size
nt = int(T / dt)            # Number of temporal steps
x = np.linspace(0, L, nx)

# Stability condition (CFL condition)
cfl = k * dt / dx
cfl_squared = cfl ** 2
if cfl > 1.0:
    raise ValueError("The simulation is unstable. Decrease dt or increase dx.")


In [25]:
# Initial conditions
def initial_condition(x):
    # Example: Gaussian pulse
    return np.exp(-k * (x - L / 2)**2)

# Initialize the solution array
u = np.zeros((nt, nx))
u[0, :] = initial_condition(x)
u[1, :] = initial_condition(x)

# Dirichlet boundary conditions: u = 0 at x = 0, L
u[:, 0] = 0
u[:, -1] = 0

In [26]:
# Initial conditions
def initial_condition(x):
    # Example: Gaussian pulse
    return np.sin(- 2 * np.pi * (x - L / 2)/L)

# Initialize the solution array
v = np.zeros((nt, nx))
v[0, :] = initial_condition(x)
v[1, :] = initial_condition(x)

# Dirichlet boundary conditions: v = 0 at x = 0, L
v[:, 0] = 0
v[:, -1] = 0

# Iterations

In [27]:
def evolve_in_time(u):
    """
    Updates the grid 'u' using a finite difference method for 'nt' time steps and 'nx' spatial points.

    Parameters:
    - u: A 2D numpy array with shape (nt, nx) representing the grid. Initial conditions should be set in u[0, :] and u[1, :].
    - cfl: The Courant-Friedrichs-Lewy number, a parameter that controls numerical stability and accuracy.
    - nt: The number of time steps to simulate.
    - nx: The number of spatial points in the grid.

    Returns:
    - u: The updated grid after 'nt' time steps.
    """
    for t in range(2, nt):  # Start from 2 because u[0, :] and u[1, :] are initial conditions
        for i in range(1, nx - 1):
            u[t, i] = 2 * (1 - cfl**2) * u[t - 1, i] - u[t - 2, i] + cfl**2 * (u[t - 1, i + 1] + u[t - 1, i - 1])
    return u


In [28]:
u = evolve_in_time(u)
v = evolve_in_time(v)

# Animate

In [29]:
# Set up the figure and axis
fig, ax = plt.subplots()
line_u, = ax.plot([], [], label='u(t, x)')  # Line for u(t, x)
line_v, = ax.plot([], [], label='v(t, x)')  # Line for v(t, x)
ax.set_xlim(0, L)
ax.set_ylim(min(np.min(u), np.min(v)), max(np.max(u), np.max(v)))  # Adjusted for both u and v

# Include legend to distinguish u and v
ax.legend()

# Animation function
def animate(t):
    line_u.set_data(x, u[t, :])  # Update line for u
    line_v.set_data(x, v[t, :])  # Update line for v
    return line_u, line_v,

# Create the animation
anim = FuncAnimation(fig, animate, frames=nt, interval=40, blit=True)

# Display the animation
plt.close()
HTML(anim.to_html5_video())